In [23]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
# import plotly.express as px
# import statsmodels.api as sm

m_per_mi = 1609.34
tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'
level = 'cntyvtd'
# level = 'county'

if level == 'county':
    sel = 'county'
else:
    sel = 'cntyvtd, county'
query = f"""
select
    {sel},
    aland,
    st_distance(polygon, (select polygon from {data_bq}.countries where country = 'Mexico')) / {m_per_mi} as dist_border,
    vap_pop / aland as density,
    vap_hisp  / vap_pop * 100 as vap_hisp_pct,
    vap_black / vap_pop * 100 as vap_black_pct,
    vap_white / vap_pop * 100 as vap_white_pct,
    (dem - rep) / votes * 100 as dem_rep_pct,
    dem,
    rep,
    votes,
    vap_hisp,
    vap_black,
    vap_white,
    vap_pop,
from (
    select
        *,
        vap_hisp + vap_black + vap_white as vap_pop,
        dem + rep as votes,
    from (
        select
            {sel},
            cast(sum(President_2020_D_Biden_general) as int) as dem,
            cast(sum(President_2020_R_Trump_general) as int) as rep,
            sum(vap_hisp_pop) as vap_hisp,
            {' + '.join([f'sum({x})' for x in get_cols(tbl) if 'vap_nonhisp' in x and 'black' in x])} as vap_black,
            {' + '.join([f'sum({x})' for x in get_cols(tbl) if 'vap_nonhisp' in x and 'white' in x and 'black' not in x])} as vap_white,
            sum(aland) / {m_per_mi**2} as aland,
            st_union_agg(polygon) as polygon,
        from
            {tbl}
        group by
            {sel}
        )
    )
where
    vap_pop > 0 and votes > 0
"""
tbl = f'{data_bq}.VRA_{level}'
# load_table(tbl=tbl, query=query)
df = read_table(tbl)
df
# print(query)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021


,cntyvtd,county,aland,dist_border,density,vap_hisp_pct,vap_black_pct,vap_white_pct,dem_rep_pct,dem,rep,votes,vap_hisp,vap_black,vap_white,vap_pop
0,025000010,Bee,1.083929,120.686560,1809.159259,78.684345,3.008669,18.306986,7.242340,385,333,718,1543,59,359,1961
1,025000003,Bee,19.599351,121.393577,486.852843,58.174387,20.121568,21.704045,-31.140802,335,638,973,5551,1920,2071,9542
2,025000017,Bee,2.654680,119.277666,511.549374,69.734904,3.387334,26.877761,-11.176471,302,378,680,947,46,365,1358
3,025000001,Bee,249.336760,121.496237,9.681685,59.113505,2.651201,38.235294,-30.171073,449,837,1286,1427,64,923,2414
4,025000018,Bee,43.240108,117.849771,42.830605,46.004320,3.023758,50.971922,-47.619048,297,837,1134,852,56,944,1852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8711,309000068,McLennan,38.145396,277.168580,34.630654,11.355034,2.043906,86.601060,-62.971429,162,713,875,150,27,1144,1321
8712,075000201,Childress,117.866355,318.990860,9.128983,20.631970,2.509294,76.858736,-77.629382,67,532,599,222,27,827,1076
8713,245000090,Jefferson,1.504504,331.657085,34.562896,21.153846,51.923077,26.923077,-24.193548,47,77,124,11,27,14,52
8714,387000005,Red River,66.044695,431.186429,17.927254,3.631757,2.280405,94.087838,-78.108108,81,659,740,43,27,1114,1184


In [24]:
df.to_csv(f'{data_path}/siam_{level}.csv')

In [17]:
df.sort_values(['vap_hisp_pct'], ascending=False).head(20)

,county,aland,dist_border,dem,rep,votes,dem_rep_pct,density,vap_hisp,vap_black,vap_white,vap_pop,vap_hisp_pct,vap_black_pct,vap_white_pct
90,Starr,1223.185145,0.000000,9099,8224,17323,5.051088,37.597742,45030,55,904,45989,97.914719,0.119594,1.965687
160,Maverick,1279.478932,0.000000,8332,6881,15213,9.537895,31.420603,38788,132,1282,40202,96.482762,0.328342,3.188896
233,Webb,3361.494411,0.000000,41842,25904,67746,23.526112,55.044268,176805,740,7486,185031,95.554258,0.399933,4.045809
79,Zavala,1297.413516,15.817968,2864,1490,4354,31.557189,5.302858,6380,45,455,6880,92.732558,0.654070,6.613372
35,Zapata,998.417285,0.000000,1826,2033,3859,-5.364084,9.573152,8837,14,707,9558,92.456581,0.146474,7.396945
204,Hidalgo,1570.964380,0.000000,128199,90527,218726,17.223375,381.741946,549880,3005,46818,599703,91.692054,0.501081,7.806864
7,Jim Hogg,1136.178713,16.714234,1197,833,2030,17.931034,3.058498,3077,4,394,3475,88.546763,0.115108,11.338129
46,Cameron,891.714704,0.000000,64063,49032,113095,13.290596,334.373762,264000,1300,32866,298166,88.541282,0.435999,11.022719
60,Brooks,943.359218,36.807569,1470,998,2468,19.124797,5.603380,4593,29,664,5286,86.889898,0.548619,12.561483
205,Willacy,590.604958,18.771709,3108,2441,5549,12.020184,25.822675,13196,447,1608,15251,86.525474,2.930955,10.543571


In [18]:
tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'
# cols = [x for x in get_cols(tbl) if 'vap_nonhisp' in x and 'race' not in x and 'pop' not in x]
cols = [x for x in get_cols(tbl) if 'vap_nonhisp' in x and 'black' in x]
query = f"""
select
    {' + '.join(cols)} as blck,
    vap_nonhisp_pop
    
from 
    {tbl}
"""
df = run_query(query=query)
df

,blck,vap_nonhisp_pop
0,758,1134
1,32,1256
2,76,1313
3,110,342
4,215,657
...,...,...
9002,10,822
9003,63,1057
9004,10,439
9005,17,1029


In [9]:
df

,vap_black,vap_nonhisp_black
0,758,731
1,32,29
2,76,71
3,110,108
4,215,207
...,...,...
9002,10,10
9003,63,62
9004,10,9
9005,17,15


In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

def get_stats(district_type):
    nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract10'
#     nodes_tbl = f'{proj_id}.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}'
    M = MCMC(random_seed=9000000, nodes_tbl=nodes_tbl)
    M.post_process1()
    M.post_process2()
    query = f"select * from {M.hash_tbl}"
    df = run_query(query)
    if district_type == 'cd':
        pop_dev = 1
        base_defect = 47
    elif district_type == 'sldu':
        pop_dev = 10
        base_defect = 26
    elif district_type == 'sldl':
        pop_dev = 10
        base_defect = 19
    loose_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect*3)
    tight_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect)
    print(f'\n\n{district_type} loose mask stats')
    display(df.loc[loose_mask, 'defect_plan'].describe())
    print(f'{district_type} tight mask stats')
    display(df.loc[tight_mask, 'defect_plan'].describe())
    return df

df_cd   = get_stats('cd')
df_sldl = get_stats('sldu')
df_sldu = get_stats('sldl')

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

nodes_opts = {
    'district_type'    : 'sldu',
    'contract_thresh'  : 10,
    'level'            : 'cntyvtd',
    'abbr'             : 'TX',
    'proposal'         : 'PLANS2101'
}

N = Nodes(**nodes_opts)
M = MCMC(nodes_tbl=N.tbl, random_seed=9000000)
M.run_chain()

# N.tbl

In [7]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *

tbl = f'{data_bq}.countries'
df = read_table(tbl)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021


In [8]:
df.query('country == "Mexico"')

,country,abbr,polygon
153,Mexico,MEX,MULTIPOLYGON(((-112.164129886 24.9968915620002...


In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *

tbl = f'{data_bq}.TX_2020_cd_planc2100_all'
query = f"""
select
    *
from (
    select
        cntyvtd,
        count(distinct county) as splits,
    from
        {tbl}
    group by 
        cntyvtd
    )
where
    splits = 1
limit
    10
"""
run_query(query)

In [16]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'

query = f"""
select
    cntyvtd,
    st_distance(polygon, (select polygon from {data_bq}.countries where country = 'Mexico')) as dist_border
from
    {tbl}
"""
df = run_query(query)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021


In [14]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'

query = f"""
select
    cntyvtd,
    st_distance(polygon, Mexico) as dist_border
from (
    select
        cntyvtd,
        polygon,
        (select polygon from {data_bq}.countries where country = 'Mexico') as Mexico
    from
        {tbl}
    )
"""
df = run_query(query)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021


In [15]:
df.head(3)

,cntyvtd,f0_
0,423000034,620146.166906
1,4990003-1,640014.881700
2,347000013,615420.378534


In [8]:
' + '.join([x for x in get_cols(tbl) if 'black' in x and 'vap_nonhisp' in x])

'vap_nonhisp_black + vap_nonhisp_white_black + vap_nonhisp_black_native + vap_nonhisp_black_asian + vap_nonhisp_black_pacific + vap_nonhisp_black_other + vap_nonhisp_white_black_native + vap_nonhisp_white_black_asian + vap_nonhisp_white_black_pacific + vap_nonhisp_white_black_other + vap_nonhisp_black_native_asian + vap_nonhisp_black_native_pacific + vap_nonhisp_black_native_other + vap_nonhisp_black_asian_pacific + vap_nonhisp_black_asian_other + vap_nonhisp_black_pacific_other + vap_nonhisp_white_black_native_asian + vap_nonhisp_white_black_native_pacific + vap_nonhisp_white_black_native_other + vap_nonhisp_white_black_asian_pacific + vap_nonhisp_white_black_asian_other + vap_nonhisp_white_black_pacific_other + vap_nonhisp_black_native_asian_pacific + vap_nonhisp_black_native_asian_other + vap_nonhisp_black_native_pacific_other + vap_nonhisp_black_asian_pacific_other + vap_nonhisp_white_black_native_asian_pacific + vap_nonhisp_white_black_native_asian_other + vap_nonhisp_white_black_

In [6]:
df

,cntyvtd,county,dist_border,density,vap_hisp_pop,vap_nonhisp_pop,vap_hisp_prop,rd_diff
0,4990003-1,Wood,397.687799,42.682627,104,1256,0.076471,71.523915
1,347000013,Nacogdoches,382.405445,19.248139,102,1313,0.072085,69.629630
2,405000003,San Augustine,417.024490,26.148020,21,342,0.057851,41.242938
3,005000013,Angelina,385.188297,1732.791614,228,657,0.257627,4.534606
4,183000003,Gregg,423.092420,1814.231429,646,4489,0.125803,39.868204
...,...,...,...,...,...,...,...,...
8711,491000150,Williamson,203.009805,2393.574613,1764,2610,0.403292,-28.257191
8712,491000491,Williamson,204.856071,1722.118233,1923,5149,0.271917,-17.012070
8713,491000263,Williamson,193.924051,2606.534897,1657,5414,0.234337,0.611087
8714,491000429,Williamson,215.163986,290.874169,1166,1243,0.484018,-21.361059


In [56]:
import geopandas as gpd
geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').simplify(0.001).buffer(0) 
type(df)

pandas.core.frame.DataFrame

In [7]:
df['density'].describe()
# df.query('density > 4000')

count     8716.000000
mean      2025.640534
std       2592.059576
min          0.007089
25%         60.455022
50%       1289.582924
75%       3296.316555
max      49865.480503
Name: density, dtype: float64

In [50]:
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV
mod_sk = RidgeCV(fit_intercept=True, scoring='r2', alphas=np.linspace(0,1,100)[1:])
mod_sk = LinearRegression(fit_intercept=True)
# mod_sk = RandomForestClassifier()
mod_sk.fit(X, y, sample_weight=w)
# print(mod_sk.coef_, mod_sk.intercept_, mod_sk.alpha_, mod_sk.best_score_)
print(mod_sk.coef_, mod_sk.intercept_, mod_sk.score(X,y))


[-3.07042343e+01 -3.31830427e-03] 69.42983082526021 0.3960337842403123


In [ ]:
0.36675915992818464

In [36]:
mod_sk.best_score_, 
mod_sk.alpha_

0.1

In [25]:
y = df['redness']
w = df['vap_pop']
X = df[['vap_hisp_prop', 'density']]
# X = df[['vap_hisp_prop']]
mod_sm = sm.WLS(exog=sm.add_constant(X), endog=y, weights=w)
res = mod_sm.fit()
print(res.summary())


                            WLS Regression Results                            
Dep. Variable:                redness   R-squared:                       0.377
Model:                            WLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     2633.
Date:                Fri, 08 Oct 2021   Prob (F-statistic):               0.00
Time:                        05:27:20   Log-Likelihood:                -39271.
No. Observations:                8716   AIC:                         7.855e+04
Df Residuals:                    8713   BIC:                         7.857e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            69.4298      0.339    204.989

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [68]:
df.shape
L = []
for i, X in df.iloc[:1000].iterrows():
    z = pd.DataFrame()
    N = X['vap_pop']
    H = X['vap_hisp_pop']
    W = N - H
    R = X['redness']
    z['cntyvtd'] = np.repeat(X['cntyvtd'], N)
    z['county'] = X['county']
    z['density'] = X['density']
    z['hisp'] = False
    z['red']  = False
    z.loc[:H, 'hisp']  = True
    z.loc[z.index[:round(H*R) ], 'red'] = True
    z.loc[z.index[-round(W*R):], 'red'] = True
    L.append(z)
Z = pd.concat(L, ignore_index=True)

In [90]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
y = Z['red']
# X = Z[['hisp', 'density']].astype(float)
# X['const'] = 1.0
X = Z[['hisp', 'density']]

# mod_sk = LinearRegression(fit_intercept=False)
mod_sk = LogisticRegression(fit_intercept=False)
# mod_sk = RandomForestClassifier()
mod_sk.fit(X, y)
# print(mod_sk.coef_, mod_sk.intercept_)

# mod_sm = sm.OLS(y, X)
# mod_sm = sm.Logit(y, X)
# res = mod_sm.fit()
# print(res.summary())
# # mod = sm.Logit(exog=sm.add_constant(X[['hisp', 'density']].astype(float)), endog=X['red'])
# res = mod.fit()
# print(res.summary())

LogisticRegression(fit_intercept=False)

In [92]:
mod_sk.score?

# (X, y)

Signature: mod_sk.score(X, y, sample_weight=None)
Docstring:
Return the mean accuracy on the given test data and labels.

In multi-label classification, this is the subset accuracy
which is a harsh metric since you require for each sample that
each label set be correctly predicted.

Parameters
----------
X : array-like of shape (n_samples, n_features)
    Test samples.

y : array-like of shape (n_samples,) or (n_samples, n_outputs)
    True labels for X.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

Returns
-------
score : float
    Mean accuracy of self.predict(X) wrt. y.
File:      /opt/conda/lib/python3.7/site-packages/sklearn/base.py
Type:      method


In [ ]:
sns.jointplot(data=data, x='vap_hisp_prop', y='redness', kind="reg");

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
import plotly.express as px

tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'
query = f"""
select
    cntyvtd,
    county,
    vap_pop,
    vap_hisp_pop / vap_pop as vap_hisp_prop,
    President_2020_R_Trump_general / votes as redness
from (
    select
        cntyvtd,
        county,
        vap_hisp_pop,
        vap_pop,
        President_2020_R_Trump_general,
        President_2020_R_Trump_general + President_2020_D_Biden_general as votes
    from
        {tbl}
    )
where
    votes > 0 and vap_pop > 0
"""
df = run_query(query)

In [ ]:
sns.lmplot?

In [ ]:
import seaborn as sns
data = df#.sample(n=50)
# ax = sns.lmplot(data=data, x='vap_hisp_prop', y='redness',
#                  scatter=False,
# #                  marker='.',
#                  ci = 68,
# #                  n_boot=5000
#                 )
# ax.set_title('Linear Regression')
sns.jointplot(data=data, x='vap_hisp_prop', y='redness', kind="reg");

In [ ]:
tips

In [ ]:
import seaborn as sns; sns.set_theme(color_codes=True)

tips = sns.load_dataset("tips")

ax = sns.regplot(x="total_bill", y="tip", data=tips)

In [ ]:
df['vap_factor'] = df['vap_pop'] / df['vap_pop'].max()
fig = px.scatter(df, x="vap_hisp_prop", y="redness", opacity=df['vap_factor'],
                 hover_data=['county', 'vap_pop']
                )
fig.show()

In [ ]:
lin.fit?

In [ ]:
sns.regplot?

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
lin = LinearRegression()
lin.fit(X=df[['vap_hisp_prop']], y=df['redness'], sample_weight=df['vap_pop'])
print(lin.coef_)
lin.fit
mod = sm.WLS(exog=sm.add_constant(df['vap_hisp_prop']), endog=df['redness'], weights=df['vap_pop'])
res = mod.fit()

print(res.summary())
sm.OLS?

In [ ]:
sm.add_constant?

In [ ]:
import statsmodels.api as sm
spector_data = sm.datasets.spector.load()
spector_data
mod = sm.OLS(spector_data.endog, spector_data.exog)

In [ ]:
spector_data.exog
spector_data.endog

In [ ]:
px.scatter?

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
import plotly.express as px

tbl = f'{data_bq}.TX_2020_cd_planc2100_cntyvtd_0_nodes'
query = f"""
select
    cntyvtd,
    county,
    vap_hisp_pop,
    vap_pop,
    President_2020_R_Trump_general,
    President_2020_R_Trump_general + President_2020_D_Biden_general as votes
from
    {tbl}
order by
    vap_pop desc
"""
df = run_query(query)
df

In [ ]:
df.query('votes < 1')

In [ ]:
fig = px.scatter(df, x="sepal_width", y="sepal_length")
fig.show()

In [ ]:
tbl = 'cmat-315920.redistricting_data.assignments_TX_2020_tabblock'
query = f"select distinct cast(geoid as int) as geoid, sldu, cd, sldl, cntyvtd from {tbl}"
df1 = run_query(query)

# file = data_path / f'assignments/TX/sldu/PLANS2101.csv'
# df2 = pd.read_csv(file)
# df2.columns = ['geoid', 'sldu']

In [ ]:
df1

In [ ]:
df2.dtypes

In [ ]:
df = df1.merge(df2, on='geoid')

In [ ]:
w = df1.groupby('cntyvtd')['cd'].nunique()
w[w>1]

Andrea - Call me

Andrea, This is a collaboration, but I feel like you have been attempt to issue increasingly direct orders to me.  We can talk out our differences, but the conversation must go both ways.  I have been working on this thing for 3 years.  My students wrote much of GerryChain.  I am an expert here.  We can talk, but I'm not okay with you attempting exert unilateral control.

The reason I have poured years of my life into this is that I believe there are features of the current map that were done to gain unfair political advantage.  As a result, I'm not willing to blindly accept the decisions that were made and expressed in the 2010 map.  I work in a county that was split even though it is far below 1 seats share.  Almost everyone agrees this was a gerrymander.  Your proposal perpetuates that.

Moreover, I ran the defect analysis you asked for on the prior congressional and senate maps.  Nearly everything in the ensemble shows defect 2-3 times what the currently enacted map has.  I'm not okay with that, especially since we can easily do better.

In [ ]:
f = N.pq
# f.with_suffix('') / 'hi.pq'
f.parent / f'{f.stem}_step10'

In [ ]:
district_type = 'sldu'
n = 31

target_pop = 29145505 / n
seats_col = f'seats_{district_type}'
results = f'cmat-315920.TX_2020_cntyvtd_{district_type}.TX_2020_cntyvtd_{district_type}_0000000_allresults'
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract2010'
defects = f'cmat-315920.redistricting_data.contract_compares_{district_type}'

query = f"""
select
    seed,
    plan,
    hash_plan,
    max(pop_deviation) as pop_deviation_plan,
    sum(whole_defect) as whole_defect_plan,
    sum(intersect_defect) as intersect_defect_plan,
    sum(defect) as defect_plan
from (
    select
        *,
        whole_defect + intersect_defect as defect
    from (
        select
            *,
            abs(cast(floor({seats_col}) - wholes as int)) as whole_defect,
            abs(cast(intersects - ceiling({seats_col}) as int)) as intersect_defect
        from (
            select
                seed,
                plan,
                hash_plan,
                cnty,
                max(pop_deviation) as pop_deviation,
                sum({seats_col}) as {seats_col},
                sum(whole) as wholes,
                count(*) as intersects
            from (
                select
                    *,
                    case when count(distinct cnty) over (partition by seed, plan, {district_type}) = 1 then 1 else 0 end as whole
                from (
                    select
                        A.seed,
                        A.plan,
                        A.hash_plan,
                        A.cnty,
                        A.{district_type},
                        max(A.pop_deviation) as pop_deviation,
                        sum(B.{seats_col}) as {seats_col}
                    from (
                        select
                            seed,
                            plan,
                            hash_plan,
                            substring(geoid, 1, 3) as cnty,
                            {district_type},
                            geoid,
                            pop_imbalance_plan as pop_deviation
                        from
                            {results}_stack
                        ) as A
                    inner join
                        {nodes} as B
                    on
                        A.geoid = B.geoid
                    group by
                        seed, plan, hash_plan, cnty, {district_type}
                    )
                )
            group by
                seed, plan, hash_plan, cnty
            )
        )
    )
group by
    seed, plan, hash_plan
"""
df = load_table(defects, query=query)

In [ ]:
query = f"""
select
    A.seed as random_seed,
    A.plan,
    A.{district_type},
    A.hash_plan,
    B.whole_defect_plan,
    B.intersect_defect_plan,
    B.defect_plan,
    B.pop_deviation_plan,
    A.* except (seed, plan, {district_type}, hash_plan, pop_imbalance_plan, nodes_plan, nodes_district)
from
    {results}_backup as A
inner join
    {output} as B
on
    A.seed = B.seed and A.plan = B.plan
"""
load_table(tbl=results, query=query)

In [ ]:

pop = 29145505
n = 38
target_pop = pop / n
district_type = 'cd'
seats_col = f'seats_{district_type}'
results_old = f'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_{district_type}_0000000_allresults_stack'
nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_{district_type}_contract2010'
output = f'cmat-315920.redistricting_data.contract_compares'

query = f"""
select
    seed,
    plan,
    max(pop_deviation) as pop_deviation,
    sum(wholes_defect) as wholes_defect,
    sum(intersects_defect) as intersects_defect,
    sum(defect) as defect
from (
    select
        *,
        wholes_defect + intersects_defect as defect
    from (
        select
            *,
            abs(cast(floor({seats_col}) - wholes as int)) as wholes_defect,
            abs(cast(intersects - ceiling({seats_col}) as int)) as intersects_defect
        from (
            select
                seed,
                plan,
                cnty,
                max(pop_deviation) as pop_deviation,
                sum({seats_col}) as {seats_col},
                sum(whole) as wholes,
                count(*) as intersects
            from (
                select
                    *,
                    case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
                from (
                    select
                        A.seed,
                        A.plan,
                        A.cnty,
                        A.cd,
                        max(A.pop_deviation) as pop_deviation,
                        sum(B.{seats_col}) as {seats_col}
                    from (
                        select
                            seed,
                            plan,
                            substring(geoid, 1, 3) as cnty,
                            cd,
                            geoid,
                            pop_imbalance_plan as pop_deviation
                        from
                            {results_old}
                        --where seed = 1126 and plan = 552
                        ) as A
                    inner join
                        {nodes} as B
                    on
                        A.geoid = B.geoid
                    group by
                        seed, plan, cnty, cd
                    )
                )
            group by
                seed, plan, cnty
            )
        )
    )
group by
    seed, plan
"""
df = load_table(output, query=query)

In [ ]:
query = f"""
select
    *
from
    {output}
"""
df = run_query(query)

In [ ]:
df['defect'].describe()

In [ ]:
mask = df['']
df['']

In [ ]:
df.sort_values('whole')

In [ ]:
df[seats_col].sum()

In [ ]:
pop = 29145505
n = 38
target_pop = pop / n
results_old = 'cmat-315920.TX_2020_cntyvtd_cd.TX_2020_cntyvtd_cd_0000000_allresults_stack'
assignments = 'cmat-315920.redistricting_data.assignments_TX_2020_tabblock'

query = f"""
select
    seed,
    plan,
    cnty,
    max(pop_deviation) as pop_deviation,
    sum(whole) as whole
from (    
    select
        *,
        case when count(distinct cnty) over (partition by seed, plan, cd) = 1 then 1 else 0 end as whole
    from (
        select
            seed,
            plan,
            cnty,
            cd,
            max(pop_deviation) as pop_deviation
        from (
            select
                seed,
                plan,
                substring(geoid, 1, 3) as cnty,
                cd,
                geoid,
                pop_imbalance_plan as pop_deviation
            from
                {results_old}
            where
                seed = 1126 and plan = 552
            )
        group by
            seed, plan, cnty, cd
        )
    )
group by
    seed, plan, cnty
"""

df = run_query(query)
#     count(*) over (partition byhash_plan,
df

In [ ]:
f = f'nodes_TX_2020_tabblock_sldu_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)

In [ ]:
f = f'nodes_TX_2020_tabblock_sldl_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)